# Simple AstrID Ingestion Test

Quick test of the data ingestion functionality without heavy dependencies.


In [ ]:
# Setup
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print("✅ Path setup complete")


In [ ]:
# Test coordinate processor
from src.domains.observations.ingestion.processors.coordinate_utils import CoordinateProcessor

coord_processor = CoordinateProcessor()

# Test random coordinate generation
ra, dec = coord_processor.get_random_coordinates(avoid_galactic_plane=True)
print(f"Random coordinates: RA={ra:.4f}°, Dec={dec:.4f}°")

# Test coordinate cleaning
test_dec = "+45:30:15.5"
clean_dec = coord_processor.clean_dec_value(test_dec)
print(f"Cleaned declination: {test_dec} -> {clean_dec:.4f}°")

print("✅ Coordinate processor working")


In [ ]:
# Test FITS processor (lightweight)
from src.adapters.imaging.fits_io import FITSProcessor

fits_processor = FITSProcessor()
print("✅ FITS processor initialized")

# You can test with actual FITS files here
print("Ready to test with FITS files from your previous project!")


In [ ]:
# Test external clients
from src.adapters.external.mast import MASTClient
from src.adapters.external.skyview import SkyViewClient

mast_client = MASTClient()
skyview_client = SkyViewClient()

print("✅ External clients initialized")
print("Ready to test with real astronomical surveys!")

# Example: Test M31 coordinates
m31_ra, m31_dec = 10.6847, 41.2690
print(f"Test target: M31 at RA={m31_ra}°, Dec={m31_dec}°")


## Ready for Testing!

The ingestion system is set up and ready. You can now:

1. **Test with your old FITS files** - Copy some FITS files from your previous project and use `fits_processor.read_fits(file_path)`

2. **Test API endpoints** - Use the running API to test ingestion endpoints

3. **Test external surveys** - Try querying MAST or SkyView for real data

4. **Visualize results** - Import matplotlib/astropy to visualize the ingested data

The notebooks are ready for your astronomical data exploration!
